In [1]:
%pip install -q ampform==0.14.* quadpy tensorwaves[jax]==0.4.*

Note: you may need to restart the kernel to use updated packages.


In [2]:
from functools import partial

import numpy as np
import qrules
import quadpy
import sympy as sp

from ampform.dynamics import (
    BlattWeisskopfSquared,
    BreakupMomentumSquared,
    PhaseSpaceFactor,
)

PDG = qrules.load_pdg()

In [3]:
# %config InlineBackend.figure_formats = ['svg']
import ampform
import attrs
import matplotlib.pyplot as plt
import numpy as np
import qrules
from ampform.dynamics import PhaseSpaceFactorSWave
from ampform.dynamics.builder import RelativisticBreitWignerBuilder
from matplotlib import cm

from tensorwaves.data import (
    SympyDataTransformer,
    TFPhaseSpaceGenerator,
    TFUniformRealNumberGenerator,
)
from tensorwaves.function.sympy import create_parametrized_function

# Modify particle masses
PDG = qrules.load_pdg()
eta_mass = PDG["eta"].mass
proton_mass = PDG["eta"].mass
resonance_mass = 0.97 * (proton_mass + eta_mass)

particle_set = set(PDG)

particle = PDG["p"]
modified_particle = attrs.evolve(particle, mass=proton_mass)
particle_set.remove(particle)
particle_set.add(modified_particle)

particle = PDG["N(1440)+"]
modified_particle = attrs.evolve(particle, mass=resonance_mass)
particle_set.remove(particle)
particle_set.add(modified_particle)

PDG = qrules.ParticleCollection(particle_set)

# Create amplitude model
reaction = qrules.generate_transitions(
    initial_state=("J/psi(1S)", [+1]),
    final_state=["p", "p~", "eta"],
    allowed_intermediate_particles=["N(1440)+"],
    allowed_interaction_types=["strong", "EM"],
    particle_db=PDG,
    mass_conservation_factor=1,
)
assert len(reaction.get_intermediate_particles()) == 1

model_builder = ampform.get_builder(reaction)
model_builder.scalar_initial_state_mass = True
model_builder.stable_final_state_ids = [0, 1, 2]
breit_wigner_builder = RelativisticBreitWignerBuilder(
    phsp_factor=PhaseSpaceFactorSWave   # <-- fixes the problem
)
for name in reaction.get_intermediate_particles().names:
    model_builder.set_dynamics(name, breit_wigner_builder)

breit_wigner_builder.energy_dependent_width = False
breit_wigner_builder.form_factor = False
model_standard_bw = model_builder.formulate()

breit_wigner_builder.energy_dependent_width = False
breit_wigner_builder.form_factor = True
model_standard_bw_with_ff = model_builder.formulate()

breit_wigner_builder.energy_dependent_width = True
breit_wigner_builder.form_factor = False
model_energy_dependent = model_builder.formulate()

breit_wigner_builder.energy_dependent_width = True
breit_wigner_builder.form_factor = True
model_energy_dependent_with_ff = model_builder.formulate()

# Generate phase space sample
rng = TFUniformRealNumberGenerator(seed=0)
phsp_generator = TFPhaseSpaceGenerator(
    initial_state_mass=reaction.initial_state[-1].mass,
    final_state_masses={i: p.mass for i, p in reaction.final_state.items()},
)
phsp_momenta = phsp_generator.generate(1_000_000, rng)
helicity_transformer = SympyDataTransformer.from_sympy(
    model_standard_bw.kinematic_variables, backend="jax"
)
phsp = helicity_transformer(phsp_momenta)
phsp = {k: v.real for k, v in phsp.items()}  # important for sqrt of Blatt-Weisskopf!

# Generate intensity distributions
func_standard_bw = create_parametrized_function(
    expression=model_standard_bw.expression.doit(),
    parameters=model_standard_bw.parameter_defaults,
    backend="jax",
)
func_standard_bw_with_ff = create_parametrized_function(
    expression=model_standard_bw_with_ff.expression.doit(),
    parameters=model_standard_bw_with_ff.parameter_defaults,
    backend="jax",
)
func_energy_dependent = create_parametrized_function(
    expression=model_energy_dependent.expression.doit(),
    parameters=model_energy_dependent.parameter_defaults,
    backend="jax",
)
func_energy_dependent_with_ff = create_parametrized_function(
    expression=model_energy_dependent_with_ff.expression.doit(),
    parameters=model_energy_dependent_with_ff.parameter_defaults,
    backend="jax",
)

# Plot 'em!
fig, ax = plt.subplots(figsize=(9, 4))

hist_kwargs = dict(
    bins=200,
    histtype="step",
    density=True,
)
ax.hist(
    np.array(phsp["m_02"].real),
    weights=np.array(func_standard_bw(phsp)),
    label="Standard Breit-Wigner",
    **hist_kwargs,
)
ax.hist(
    np.array(phsp["m_02"].real),
    weights=np.array(func_standard_bw_with_ff(phsp)),
    label="Standard Breit-Wigner with form factor",
    linestyle="dotted",
    **hist_kwargs,
)
ax.hist(
    np.array(phsp["m_02"].real),
    weights=np.array(func_energy_dependent(phsp)),
    label="Energy-dependent Breit-Wigner",
    **hist_kwargs,
)
ax.hist(
    np.array(phsp["m_02"].real),
    weights=np.array(func_energy_dependent_with_ff(phsp)),
    label="Energy-dependent Breit-Wigner with form factor",
    linestyle="dotted",
    **hist_kwargs,
)

ax.set_yscale("log")
ax.set_xlabel(R"$m_{p\eta}$ [GeV]")

resonances = sorted(
    reaction.get_intermediate_particles(),
    key=lambda p: p.mass,
)
evenly_spaced_interval = np.linspace(0, 1, len(resonances))
colors = [cm.rainbow(x) for x in evenly_spaced_interval]
for p, color in zip(resonances, colors):
    ax.axvline(x=p.mass, linestyle="dotted", label=p.name, color=color)
ax.legend()
plt.show()

Propagating quantum numbers:   0%|          | 0/48 [00:00<?, ?it/s]

ImportError: Module tensorflow not installed. Reinstall tensorwaves with:

  pip install tensorwaves[tf]


In [ ]:
def na2(s, m1, m2, L, q0):
    q_squared = BreakupMomentumSquared(s, m1, m2)
    return BlattWeisskopfSquared(
        z=q_squared / (q0**2),
        angular_momentum=L,
    )


L = 2
s, m1, m2 = sp.symbols("s m1 m2", real=True)
epsilon = sp.Symbol("epsilon", positive=True)
s_prime = sp.Symbol(R"s^{\prime}", real=True)

q0 = sp.Symbol("q0", real=True)
s_thr = (m1 + m2) ** 2
integrand = (
    PhaseSpaceFactor(s_prime, m1, m2) * na2(s_prime, m1, m2, L, q0)
) / ((s_prime - s_thr) * (s_prime - s - epsilon * sp.I))
integrand

BlattWeisskopfSquared(2, BreakupMomentumSquared(s^{\prime}, m1, m2)/q0**2)*PhaseSpaceFactor(s^{\prime}, m1, m2)/((s^{\prime} - (m1 + m2)**2)*(-I*epsilon - s + s^{\prime}))

In [ ]:
integrand_func = sp.lambdify(
    args=(s_prime, s, epsilon, m1, m2, q0),
    expr=integrand.doit(),
    modules="numpy",
)

In [ ]:
%pip install -q quadpy

Note: you may need to restart the kernel to use updated packages.


In [ ]:
@np.vectorize
def vectorized_quad(func, a, b, **func_kwargs):
    values, _ = quadpy.quad(partial(func, **func_kwargs), a, b)
    return values

In [ ]:
s_min, s_max = -0.15, 1.4
s_domain = np.linspace(s_min, s_max, num=50)

In [ ]:
m1_val = PDG["pi0"].mass
m2_val = PDG["eta"].mass
s_thr_val = float(s_thr.subs({m1: m1_val, m2: m2_val}))
integral_value = vectorized_quad(
    integrand_func,
    a=s_thr_val,
    b=np.inf,
    s=s_domain,
    epsilon=1e-3,
    m1=m1_val,
    m2=m2_val,
    q0=1.0,
)
integral_value

array([0.02149997+2.34976267e-06j, 0.021575  +2.39410036e-06j,
       0.02165146+2.44072113e-06j, 0.02172944+2.48985230e-06j,
       0.02180901+2.54175752e-06j, 0.02189027+2.59674521e-06j,
       0.02197333+2.65517959e-06j, 0.0220583 +2.71749557e-06j,
       0.0221453 +2.78421888e-06j, 0.02223449+2.85599443e-06j,
       0.02232605+2.93362692e-06j, 0.02242016+3.01814111e-06j,
       0.02251708+3.11087473e-06j, 0.02261708+3.21362882e-06j,
       0.02272052+3.32892580e-06j, 0.02282785+3.46048858e-06j,
       0.02293968+3.61423036e-06j, 0.02305685+3.80064418e-06j,
       0.0231807 +4.04232011e-06j, 0.02331387+4.41505201e-06j,
       0.02346414+1.06665934e-05j, 0.02362069+3.51329782e-05j,
       0.02377729+7.19677549e-05j, 0.02393282+1.19096483e-04j,
       0.02408646+1.75249336e-04j, 0.02423758+2.39431771e-04j,
       0.02438575+3.10799126e-04j, 0.02453062+3.88614125e-04j,
       0.02467198+4.72220957e-04j, 0.02480966+5.61035555e-04j,
       0.02494356+6.54535110e-04j, 0.02507363+7.5225172